# Tensorflow (No Keras!) Implementation
Documentation: https://www.tensorflow.org/api_docs/python/tf

In [14]:
# Install dependencies
!pip install tensorflow tensorflow-datasets

Note: you may need to restart the kernel to use updated packages.


In [16]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [17]:
# Load MNIST data using TensorFlow Datasets
ds_train, ds_test = tfds.load('mnist', split=['train', 'test'], as_supervised=True, shuffle_files=True)

In [18]:
# Hyperparameters
learning_rate = 0.001
batch_size = 32
epochs = 10

In [19]:
# Normalize the images and prepare the datasets
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

train_data = ds_train.map(normalize_img).shuffle(10000).batch(batch_size)
test_data = ds_test.map(normalize_img).batch(batch_size)

In [20]:
# Define the model using low-level TensorFlow operations
class SimpleNN(tf.Module):
    def __init__(self):
        # Initialize weights and biases
        self.weights1 = tf.Variable(tf.random.normal([784, 128], stddev=0.1))
        self.biases1 = tf.Variable(tf.zeros([128]))
        self.weights2 = tf.Variable(tf.random.normal([128, 10], stddev=0.1))
        self.biases2 = tf.Variable(tf.zeros([10]))

    def __call__(self, x):
        x = tf.reshape(x, [-1, 784])  # Flatten the image
        x = tf.add(tf.matmul(x, self.weights1), self.biases1)
        x = tf.nn.relu(x)
        x = tf.add(tf.matmul(x, self.weights2), self.biases2)
        return x

model = SimpleNN()

In [21]:
# Define loss function
def loss_fn(logits, labels):
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels))

In [22]:
# Define optimizer
optimizer = tf.optimizers.Adam(learning_rate)

In [23]:
# Training function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        logits = model(images)
        loss = loss_fn(logits, labels)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Training loop
for epoch in range(epochs):
    for images, labels in train_data:
        loss = train_step(images, labels)
    print(f'Epoch {epoch + 1}, Loss: {loss.numpy()}')

Epoch 1, Loss: 0.21739478409290314
Epoch 2, Loss: 0.24804949760437012
Epoch 3, Loss: 0.10593440383672714
Epoch 4, Loss: 0.036454398185014725
Epoch 5, Loss: 0.00372866028919816
Epoch 6, Loss: 0.007430126890540123
Epoch 7, Loss: 0.0308841485530138
Epoch 8, Loss: 0.005525527987629175
Epoch 9, Loss: 0.000722321798093617
Epoch 10, Loss: 0.02371489815413952


In [24]:
# Evaluation function
def evaluate(dataset):
    accuracy_metric = tf.metrics.Accuracy()
    for images, labels in dataset:
        logits = model(images)
        predictions = tf.argmax(logits, axis=1, output_type=tf.int64)
        accuracy_metric.update_state(labels, predictions)
    return accuracy_metric.result().numpy()

# Evaluate the model
accuracy = evaluate(test_data)
print(f'Test accuracy: {accuracy}')

Test accuracy: 0.9772364497184753
